In [1]:
import os
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import yaml
import itertools
from scipy.interpolate import UnivariateSpline

In [30]:
total=0
for i in range(0,8):
    print(i)
    GICC21_ties = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/GICC21 matchpoints.xlsx', sheet_name=2, usecols=[7+(i*2), 9+(i*2)], skiprows=1)
    GICC21_ties.dropna(inplace=True)
    print(len(GICC21_ties))
    total+=len(GICC21_ties)

print(f"Total: {total}")

0
290
1
290
2
290
3
153
4
153
5
290
6
24
7
18
Total: 1508


In [38]:
file_dir_AICC2023 = '/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/AICC2023-LowRes'

list_sites = ['EDC','TALDICE','VK','EDML','NGRIP']
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

pairs_antarctic = []
for pair in pairs:
    if pair.split('-')[0] != 'NGRIP' and pair.split('-')[1] != 'NGRIP':
        pairs_antarctic.append(pair)

pairs_bipolar = []
for pair in pairs:
    if pair.split('-')[0] == 'NGRIP' or pair.split('-')[1] == 'NGRIP':
        pairs_bipolar.append(pair)

synchro = 'iceice_synchro_horizons.txt'

antarctic_count = pd.DataFrame(columns=['depth1', 'depth2'])
bipolar_count = pd.DataFrame(columns=['depth1', 'depth2'])

for pair in pairs_antarctic:
    file_to_check = os.path.join(file_dir_AICC2023, pair, synchro)
    if os.path.exists(file_to_check):
        df = pd.read_csv(file_to_check, comment='#', sep='\t')
        antarctic_count = pd.concat([antarctic_count, df])
        antarctic_count.reset_index()

for pair in pairs_bipolar:
    file_to_check = os.path.join(file_dir_AICC2023, pair, synchro)
    if os.path.exists(file_to_check):
        df = pd.read_csv(file_to_check, comment='#', sep='\t')
        df = df[['depth1', 'depth2']]
        bipolar_count = pd.concat([bipolar_count, df])
        bipolar_count.reset_index()

print(f'total antarctic: {len(antarctic_count)}')
print(f'total bipolar {len(bipolar_count)}')


total antarctic: 381
total bipolar 166


/var/folders/pq/tk8k94gn22v6mr07byx05hzc0000gn/T/ipykernel_40322/2255846560.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  antarctic_count = pd.concat([antarctic_count, df])
/var/folders/pq/tk8k94gn22v6mr07byx05hzc0000gn/T/ipykernel_40322/2255846560.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bipolar_count = pd.concat([bipolar_count, df])
